<a href="https://colab.research.google.com/github/nfe9r6/Portfolio/blob/main/Case_Competition_Data%20Cleaning_Testing%20Forward%20fill.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Competition Template

## Import Libraries

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from google.colab import drive

## Import Dataset

In [ ]:
import os
os.chdir('/content/drive/My Drive/Humana Mays Competition')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import os
os.chdir('/content/drive/My Drive/Humana Mays Competition')

In [ ]:
df = pd.read_csv('Competition_Data.csv')
df_clean = pd.read_csv('Competition_Data.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (80,193) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
print(df.shape[0])
print(df_clean.shape)

69572
(69572, 826)


## Prepare/Preprocess Data

### Categorical Data

Created a list of categorical columns and their variables

In [ ]:
#Create the list
object_list = []

for column_name in df.columns:
  if df[column_name].dtype == 'object':
    object_list.append(column_name)

object_list = object_list[1:] #Removed 'person_id_syn'. It's the patient identifier - not needed in analysis

print("There are {} categorical columns.\n".format(len(object_list)))

#Display the list and the column variables
for i in object_list:
  variables = []
  temp_list = df[i]
  for j in temp_list:
    if j not in variables:
      variables.append(j)
  print("{}: {}".format(i, variables))


There are 21 categorical columns.

src_platform_cd: ['EM', 'LV']
sex_cd: ['F', 'M']
lang_spoken_cd: ['ENG', 'SPA', 'E']
mabh_seg: ['UNK', 'C2', 'H6', 'H2', 'H3', 'H4', 'H1', 'C4', 'C5', 'H7', 'H5', 'C1', 'C7', 'C3', 'H8', 'C6']
cms_ra_factor_type_cd: [nan, 'CP', 'CN', 'CF', 'E', 'SE', 'D', 'C2', 'I', '*']
cons_cmys: [nan, '4', '6', '3', '5', '2', '*', nan, 3.0, nan, nan, 4.0, 5.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 6.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, 2.0, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, nan, n

In [ ]:
y_n_columns = ['hedis_dia_eye', 'hedis_dia_hba1c_ge9', 'hedis_dia_hba1c_test', 'hedis_dia_ldc_c_control', 'hedis_dia_ldc_c_screen', 
               'hedis_dia_ma_nephr']

### Missing Data

Function creates a list of columns with missing data. 

In [ ]:
def null_columns(dataframe):
  columns_with_nulls = []
  for i in dataframe.columns:
    if dataframe[i].isnull().values.any():
      columns_with_nulls.append(i)
  return columns_with_nulls

Function displays columns with null values and how many.

In [ ]:
def null_columns_with_counts(dataframe):
  for i in null_columns(dataframe):
    null_values = dataframe[i].isnull().sum()
    print("[{}]: {} null values".format(i, null_values))

Function displays the number of null columns in datafame.

In [ ]:
def nulls_count(dataframe):
  print("There are {} columns with missing data.".format(len(null_columns(dataframe))))

nulls_count(df) #131 columns missing data in original dataframe

There are 131 columns with missing data.


#### Eliminating Rows/Colums

In [ ]:
#231 rows were missing almost 100 pieces of data in original dataframe. This eliminated those rows and most the columns missing data
list_of_null_rows = []
for i in range(0, df.shape[0]):
  if pd.isnull(df['credit_bal_autobank'][i]):
    list_of_null_rows.append(df.iloc[i, :])
    df_clean.drop([i],axis =0, inplace = True)

nulls_df = pd.DataFrame(data=list_of_null_rows, columns=df.columns)

#Reset the index on df_clean
df_clean.reset_index(drop=True, inplace=True)

In [ ]:
nulls_count(df_clean) #38 columns in original dataframe after this step

There are 38 columns with missing data.


In [ ]:
#Created separate dataframe for 19,088 rows missing consumer data
nineteen_k_list = []
count = 0
for i in range(0, df_clean.shape[0]):
  if pd.isnull(df_clean['cons_homstat'][i]):
    nineteen_k_list.append(df.iloc[i, :])
    df_clean.drop([i],axis =0, inplace = True)

nineteen_k_df = pd.DataFrame(data=nineteen_k_list, columns=df_clean.columns)

#Reset the index of df_clean
df_clean.reset_index(drop=True, inplace=True)

In [ ]:
nulls_count(df_clean) #27 columns missing data

There are 38 columns with missing data.


In [ ]:
#These 3 columns were missing over 54,000 pieces of data in original dataframe
columns_to_remove = ['hedis_ami', 'hedis_cmc_ldc_c_control', 'hedis_cmc_ldc_c_screen']
dropped_columns = []
dropped_columns.append(df[map(lambda x: x, columns_to_remove)])
df_clean.drop(columns_to_remove, axis=1, inplace=True)

In [ ]:
nulls_count(df_clean) #24 columns missing data

There are 35 columns with missing data.


In [ ]:
null_columns_with_counts(df_clean)

[cms_ma_risk_score_nbr]: 2631 null values
[cms_partd_ra_factor_amt]: 2658 null values
[cms_ra_factor_type_cd]: 2934 null values
[cms_risk_adj_payment_rate_a_amt]: 2597 null values
[cms_risk_adj_payment_rate_b_amt]: 2598 null values
[cms_risk_adjustment_factor_a_amt]: 2599 null values
[cms_rx_risk_score_nbr]: 2598 null values
[cms_tot_ma_payment_amt]: 2598 null values
[cms_tot_partd_payment_amt]: 2598 null values
[cons_cmys]: 9136 null values
[cons_hcaccprf_h]: 9139 null values
[cons_hcaccprf_p]: 9138 null values
[cons_hhcomp]: 9140 null values
[cons_homstat]: 9140 null values
[cons_n2029_y]: 9139 null values
[cons_n2mob]: 9138 null values
[cons_n2pbl]: 9138 null values
[cons_n2pmv]: 9138 null values
[cons_n65p_y]: 9139 null values
[cons_online_buyer]: 9137 null values
[cons_ret_y]: 9139 null values
[cons_retail_buyer]: 9139 null values
[cons_veteran_y]: 9138 null values
[credit_bal_1stmtg_30to59dpd]: 2 null values
[credit_bal_1stmtg_severederog]: 3 null values
[credit_bal_agencyfirstmt

In [ ]:
#These were the columns that appeared to be float values. Filling these null values with their respective means
floats_with_nulls = ['cms_ma_risk_score_nbr', 'cms_partd_ra_factor_amt', 'cms_risk_adj_payment_rate_a_amt', 'cms_risk_adj_payment_rate_b_amt',
                     'cms_risk_adjustment_factor_a_amt', 'cms_rx_risk_score_nbr', 'cms_tot_ma_payment_amt', 'cms_tot_partd_payment_amt',
                     'credit_bal_1stmtg_30to59dpd', 'credit_bal_1stmtg_severederog', 'credit_bal_agencyfirstmtg_60dpd', 
                     'credit_hh_totalallcredit_severederog', 'credit_num_consumerfinance', 'credit_num_nonagn1stmorg_bankruptc']

for i in floats_with_nulls:
  df_clean[i] = df_clean[i].fillna(df[i].mean())

---

In [ ]:
nulls_count(df_clean) #13 columns missing data

There are 35 columns with missing data.


In [ ]:
null_columns_with_counts(df_clean)

[cms_ra_factor_type_cd]: 1767 null values
[cons_hcaccprf_h]: 1 null values
[cons_hhcomp]: 4 null values
[cons_n2029_y]: 1 null values
[cons_n65p_y]: 1 null values
[cons_ret_y]: 1 null values
[cons_retail_buyer]: 2 null values
[hedis_dia_eye]: 598 null values
[hedis_dia_hba1c_ge9]: 599 null values
[hedis_dia_hba1c_test]: 601 null values
[hedis_dia_ldc_c_control]: 599 null values
[hedis_dia_ldc_c_screen]: 599 null values
[hedis_dia_ma_nephr]: 599 null values


In [ ]:
df_clean[map(lambda x: x, null_columns(df_clean))].head(10)

In [ ]:
#These columns were found to be categorical and missing column data. Filling these with forward fill 
categorical_with_nulls = ['cms_ra_factor_type_cd', 'cons_hhcomp', 'cons_homstat', 'hedis_dia_eye', 'hedis_dia_hba1c_ge9', 
                          'hedis_dia_hba1c_test', 'hedis_dia_ldc_c_control', 'hedis_dia_ldc_c_screen', 'hedis_dia_ma_nephr',
                          'cons_hcaccprf_h', 'cons_n2029_y', 'cons_n65p_y', 'cons_ret_y', 'cons_retail_buyer']

for i in categorical_with_nulls:
  df_clean[i] = df_clean[i].fillna(method='ffill')

In [ ]:
nulls_count(df_clean)

There are 0 columns with missing data.


In [ ]:
#% of transportation issues in original dataframe
df.transportation_issues.sum()/df.shape[0]

0.1465675846604956

In [ ]:
#% of transportation issues in clean dataframe
df_clean.transportation_issues.sum()/df_clean.shape[0]

0.1292659144727678

In [ ]:
#% of transportation issues in 19,000 removed records
nineteen_k_df.transportation_issues.sum()/nineteen_k_df.shape[0]

0.14255029337803857

In [ ]:
#% of transportation in 231 removed records
nulls_df.transportation_issues.sum()/nulls_df.shape[0]

0.22077922077922077

In [ ]:
df.transportation_issues.sum()

10197

In [ ]:
df_clean.shape[0]

50253

In [ ]:
df_clean.transportation_issues.sum()

6496

In [ ]:
nineteen_k_df.shape[0]

19088

In [ ]:
nineteen_k_df.transportation_issues.sum()

2721

## Split Training/Test Set

In [ ]:
#X = df.iloc[:, :-1].values     #independent variables
#y = df.iloc[:, -1].values      #depedent variable

In [ ]:
#from sklearn.model_selection import train_test_split
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)